In [7]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cohen
from itertools import combinations


In [4]:
url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/iaa_050123.0913.tsv'
df = pd.read_csv(url, sep = '\t')

In [8]:
df

,Unnamed: 0.1,Unnamed: 0,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,keyword,normalized_roberta_polarity,textblob_label,vader_label,AFINN_label,SentiWordNet_label,roberta_label,our_label_sarik,our_label,our_label_anu
0,0,13190,1633665401883332608,en,everyone,3/9/23 3:05,rt eladnehorai too many liberals are sleeping...,RT @EladNehorai: Too many liberals are sleepin...,2.360557e+09,"[{'type': 'retweeted', 'id': '1633632434972663...",...,obama,0.072673,positive,positive,neutral,neutral,negative,negative,negative,negative
1,1,7555,1633965977665966080,en,everyone,3/9/23 23:00,chatgpt msi a520ma pro am4 micro atx motherboa...,ChatGPT: MSI A520M-A Pro AM4 Micro ATX Motherb...,6.791062e+06,NaN,...,chatgpt,0.633470,neutral,neutral,neutral,neutral,positive,neutral,neutral,neutral
2,2,1728,1633219223626432512,en,everyone,3/7/23 21:33,rt sagesurge marlon wayans roasting chris roc...,RT @sagesurge: Marlon Wayans roasting Chris Ro...,3.180721e+08,"[{'type': 'retweeted', 'id': '1632525319973289...",...,#chrisrocklive,0.277563,neutral,neutral,neutral,neutral,negative,negative,neutral,neutral
3,3,29831,1633874107552108544,en,everyone,3/9/23 16:55,rt visegrad24 its time for the west to send p...,RT @visegrad24: It’s time for the West to send...,1.497060e+18,"[{'type': 'retweeted', 'id': '1633639497408696...",...,ukraine,0.429270,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
4,4,1632,1633227958260473856,en,everyone,3/7/23 22:07,rt sagesurge marlon wayans roasting chris roc...,RT @sagesurge: Marlon Wayans roasting Chris Ro...,1.522840e+18,"[{'type': 'retweeted', 'id': '1632525319973289...",...,#chrisrocklive,0.277563,neutral,neutral,neutral,neutral,negative,negative,neutral,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,3425,1633095016355622912,en,everyone,3/7/23 13:19,rt tasterreblanche sagesurge he also said #c...,RT @Tasterreblanche: @sagesurge He also said🤷🏻...,5.078723e+08,"[{'type': 'retweeted', 'id': '1632830373305409...",...,#chrisrocklive,0.422684,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
96,96,26912,1633876536695586816,en,everyone,3/9/23 17:04,rt martinlzinn aaronjmate incredible syria i...,RT @MartinLZinn: @aaronjmate Incredible Syria ...,9.712790e+17,"[{'type': 'retweeted', 'id': '1633754771147444...",...,ukraine,0.018632,neutral,negative,neutral,neutral,negative,negative,negative,negative
97,97,20013,1633670596663054336,en,everyone,3/9/23 3:26,rt visegrad24 breakingrussia has fired a wave...,RT @visegrad24: BREAKING:\n\nRussia has fired ...,2.477166e+09,"[{'type': 'retweeted', 'id': '1633637043258499...",...,russia,0.396495,neutral,negative,neutral,negative,neutral,negative,negative,neutral
98,98,26101,1633877201052667904,en,everyone,3/9/23 17:07,dont miss out on the uefa europa league game b...,Don't miss out on the UEFA Europa League game ...,1.004140e+18,NaN,...,ukraine,0.962715,neutral,positive,neutral,neutral,positive,positive,positive,neutral


#### Using the agreement package in NLTK to calculate agreement metrics on a set of annotations (For more than 2 annotators use Fleiss or Krippendorf's Alpha )

In [17]:
from  nltk.metrics import agreement

In [19]:
# Reformat the data into the form AnnotationTask expects.
#COHEN's WORKS ONLY FOR 2 annotators!

data = []
for idx, row in df.iterrows():
    data.append(("a1", idx, row["our_label_sarik"]))
    data.append(("a2", idx, row["our_label_anu"]))
    data.append(("a3", idx, row["our_label_mel"]))

    
atask = agreement.AnnotationTask(data=data)

#print("Cohen's Kappa:", atask.kappa())
print("Fleiss's Kappa:", atask.multi_kappa())

Fleiss's Kappa: 0.4351168994194257


In [31]:
# This function maps labels into a numeric space,
#. so we can rely on the ordering of labels.
def priority_distance(left_label, right_label):
    mapped_labels = {
        "positive": 1,
        "neutral": 2,
        "negative": 3,
    }
    left_i = mapped_labels[left_label]
    right_i = mapped_labels[right_label]
    
    return abs(left_i - right_i)

# Create a new annotation task with the distance function
atask_2 = agreement.AnnotationTask(data=data, distance=priority_distance)
print("Krippendorf's Alpha:", atask_2.alpha())

Krippendorf's Alpha: 0.4920179267113859


#### Fliess Kappa using StatsModel

In [ ]:
!pip install statsmodels

In [53]:
import numpy as np
from scipy import stats
import statsmodels
from statsmodels.stats import inter_rater as irr

In [72]:
final_table = []

list_sarik = df['our_label_sarik']
list_anu = df['our_label_anu']
list_mel = df['our_label_mel']
final_table.append(list_sarik)
final_table.append(list_anu)
final_table.append(list_mel)

In [68]:
final_table[2]

0     negative
1      neutral
2      neutral
3      neutral
4      neutral
        ...   
95     neutral
96    negative
97    negative
98    positive
99    negative
Name: our_label_mel, Length: 100, dtype: object

In [75]:
#Extra don't really need to do
dats, cats = irr.aggregate_raters(final_table)

#Dats is the category counts and cats is the categories
print(dats,cats)

[[46 36 18]
 [25 60 15]
 [34 43 23]] ['negative' 'neutral' 'positive']


In [ ]:
#To have subjects as rows and raters as columns
giro = np.array(final_table).transpose()
print(giro)

In [74]:
score = irr.fleiss_kappa(irr.aggregate_raters(giro)[0], method='fleiss')
score

0.42673130684029875

####krippendorff score
needs numbers: doesn't work with 'neutral', 'negative', 'positive'

In [79]:
!pip install krippendorff


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
array = np.array(final_table)
print(array)

[['negative' 'neutral' 'negative' 'neutral' 'negative' 'neutral'
  'neutral' 'neutral' 'negative' 'negative' 'positive' 'neutral'
  'neutral' 'neutral' 'negative' 'negative' 'negative' 'negative'
  'negative' 'negative' 'positive' 'negative' 'positive' 'negative'
  'positive' 'negative' 'neutral' 'negative' 'positive' 'negative'
  'negative' 'negative' 'negative' 'neutral' 'neutral' 'positive'
  'neutral' 'neutral' 'neutral' 'positive' 'negative' 'negative'
  'negative' 'negative' 'positive' 'neutral' 'negative' 'neutral'
  'negative' 'negative' 'neutral' 'negative' 'neutral' 'neutral'
  'neutral' 'negative' 'negative' 'positive' 'neutral' 'positive'
  'positive' 'positive' 'positive' 'negative' 'neutral' 'neutral'
  'positive' 'negative' 'negative' 'negative' 'neutral' 'neutral'
  'neutral' 'neutral' 'positive' 'neutral' 'negative' 'negative'
  'neutral' 'negative' 'negative' 'positive' 'neutral' 'neutral'
  'positive' 'neutral' 'negative' 'negative' 'neutral' 'negative'
  'negative' 

In [84]:
import krippendorff as kd

kd.alpha(array, level_of_measurement='nominal')


ValueError: ignored

References:
1. https://gist.github.com/cbuntain/9dd7e42d5d8ab34609162410e06f3270
2. https://stackoverflow.com/questions/56481245/inter-rater-reliability-calculation-for-multi-raters-data

